In [ ]:
import requests
import json
import re

In [ ]:
'''
网址：https://voice.baidu.com/act/newpneumonia/newpneumonia/?from=osari_aladin_banner#tab4
可以爬取历史数据
在Network的JS中，有国内和世界的历史数据，可以爬取作为数据分析和数据可视化用的
'''

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
}
#下面的网址是变化的，怎么自动获取现在尚不清楚
request_url = 'https://voice.baidu.com/newpneumonia/get?target=trend&isCaseIn=1&stage=publish&callback=jsonp_1609333474158_99551'
'''
针对jsonp的修改的方法：先转为字符串，然后把那个时间戳去掉，最后把圆括号去掉，使其符合json格式
'''
text = requests.get(request_url, headers=headers)
text = text.text
# text = text.replace('jsonp_1609240013572_58685', '')
text = re.sub('jsonp_[0-9]{13}_[0-9]{5}', '', text) #正则表达式
text = text[1:-2]
# print(text)
results = json.loads(text)

In [ ]:
Date = results['data'][209]['trend']['updateDate']

In [ ]:
import pandas as pd

In [ ]:
DatePd = pd.DataFrame(Date, columns=['country'])

In [ ]:
Country = []
for name in results['data']:
    Country.append(name['name'])
Country

In [ ]:
dayNum = len(Date)

In [ ]:
FinalDataFrame = DatePd

In [ ]:
for country in results['data']:
    dayStart = dayNum - len(country['trend']['list'][0]['data'])
    totalCasesToday = []
    for _ in range(dayStart):
        totalCasesToday.append(0)
    for case in country['trend']['list'][0]['data']:
        totalCasesToday.append(case)
    countryCasesFrame = pd.DataFrame(totalCasesToday)
    countryCasesFrame.columns = [country['name']]
    FinalDataFrame = pd.concat([FinalDataFrame, countryCasesFrame], axis=1)

In [ ]:
FinalData = pd.DataFrame(FinalDataFrame.values.T, index=FinalDataFrame.columns, columns=FinalDataFrame.index)
FinalData

In [ ]:
import json

In [ ]:
with open('国家与洲.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [ ]:
from collections import defaultdict

In [ ]:
dict = defaultdict(int)

In [ ]:
for name in json_data:
    dict[name['country_cname']] = name['continent_cname']

In [ ]:
Continent = ['Continent']
for country in Country:
    if dict[country] == 0:
        Continent.append('未确定')
    else:
        Continent.append(dict[country])
Continent

In [ ]:
# dataContinent = pd.DataFrame(Continent, columns=['Continent'])
col_name = FinalData.columns.tolist()
col_name.insert(0, 'Continent')
FinalData = FinalData.reindex(columns=col_name)
FinalData['Continent'] = Continent

In [ ]:
FinalData

In [ ]:
FinalData.to_csv('WorldCasesCovid19.csv', header=False)

### 这个程序现有的问题是如何把国内的数据添加到这儿，思路是把各省的数据统计起来

### 接下来可以利用现有的可视化工具，比如Flourish等，可以快速得到动态可视化结果，也可以直接编写程序